In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("VideoGamesSales.csv")

In [15]:
#check for and remove duplicate rows
#df=df[df.duplicated]
df = df.drop_duplicates()

#Replace null values in Publisher and Region
df["Publisher"] = df["Publisher"].fillna("Unknown")
df["Region"] = df["Region"].fillna("Unknown")

#fix NA_sales type
df['NA_Sales'] = df['NA_Sales'].replace('[$]','',regex=True)
df['NA_Sales'] = pd.to_numeric(df['NA_Sales'], errors = 'coerce')
Average_Sales = df['NA_Sales'].mean()

#Catagorize United States column correctly
df['Country']=df['Country'].replace({'USA': 'United States'})
df['Country']=df['Country'].str.title()

#rename columns in clean format
df=df.rename(columns={'NA_Sales':'National Sales', 'Global_Sales':'Global Sales','NA_Profit':'National Profit','Global_Profit':'Global Profit'})

df.head(2)

,Rank,Name,Platform,Year,Month,Genre,Publisher,Country,City,State,Region,National Sales,Global Sales,National Profit,Global Profit
0,1,Wii Sports,Wii,2010,Jan,Sports,Nintendo,United States,Fairfield,California,West,41.49,82.74,12.447,24.822
1,2,Super Mario Bros.,NES,2010,Feb,Platform,Nintendo,United States,Edmonds,Washington,West,29.08,40.24,8.724,12.072
